<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [12]:
# version de python
import platform
platform.python_version()

'3.8.5'

In [13]:
# la liste des packages installés
!conda list

# packages in environment at /home/lab/anaconda3/envs/azure:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
adal                      1.2.4                    pypi_0    pypi
applicationinsights       0.11.9                   pypi_0    pypi
async_generator           1.10                       py_0    conda-forge
attrs                     20.2.0             pyh9f0ad1d_0    conda-forge
azure-common              1.1.25                   pypi_0    pypi
azure-core                1.8.1                    pypi_0    pypi
azure-graphrbac           0.61.1                   pypi_0    pypi
azure-identity            1.2.0                    pypi_0    pypi
azure-mgmt-authorization  0.61.0                   pypi_0    pypi
azure-mgmt-containerregistry 2.8.0                    pypi_0    pypi
azure-mgmt-keyvault       2.2.0                    pypi_0    pypi
azure-mgmt-resource       10.2.0                   pypi_0    pypi
a

In [14]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.13.0


Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [15]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

jt-dp100 loaded


# Avec les datastores

In Azure ML, datastores are references to storage locations, such as Azure Storage blob containers. Every workspace has a default datastore - usually the Azure storage blob container that was created with the workspace. If you need to work with data that is stored in different locations, you can add custom datastores to your workspace and set any of them to be the default.

[datastore dans la SDK](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py)

## View Datastores

Run the following code to determine the datastores in your workspace:

In [16]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspacefilestore - Default = False
workspaceblobstore - Default = True


## Upload Data to a Datastore

In [17]:
default_ds.upload_files(files=['data/titanic/train.csv'], # Upload the csv files in /data
                       target_path='titanic-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Uploading an estimated of 1 files
Uploading data/titanic/train.csv
Uploaded data/titanic/train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_f14997add3e2492cafe6d061ff3b58f5

In [18]:
data_ref = default_ds.path('titanic-data').as_download(path_on_compute='titanic-data')
print(data_ref)

$AZUREML_DATAREFERENCE_1a8c824b447c433cac6abf5cbe9e3489


Pour utiliser cette référence, il faut passer par un script...

## création d'un script

In [19]:
import os, shutil

# Create a folder for the experiment files
script_folder_name = 'script/3-titanic-files'
experiment_folder = './' + script_folder_name
os.makedirs(script_folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/titanic/train.csv', os.path.join(script_folder_name, "titanic.csv"))

'script/3-titanic-files/titanic.csv'

In [20]:
%%writefile $script_folder_name/titanic-data.py

from azureml.core import Run
import pandas as pd

# Get the args
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder reference')
args = parser.parse_args()

# Get the experiment run context
run = Run.get_context()


# load the titanic data from the data reference
data_folder = args.data_folder
print("Loading data from", data_folder)
# Load all files and concatenate their contents as a single dataframe
all_files = os.listdir(data_folder)
titanic = pd.concat((pd.read_csv(os.path.join(data_folder,csv_file)) for csv_file in all_files))


# Count the rows and log the result
row_count = (len(titanic))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Save a sample of the data and upload it to the experiment output
titanic.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = 'sample.csv')

# Complete the run
run.complete()

Writing script/3-titanic-files/titanic-data.py


In [21]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set up the parameters
script_params = {
    '--data-folder': data_ref # data reference to download files from datastore
}


# Create an estimator
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='titanic-data.py',
                    script_params=script_params,
                    compute_target = 'local'
                   )

# Create an experiment
experiment_name = 'titanic-data'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'titanic-data_1600263529_0aa75e48',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-09-16T13:39:04.065038Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'c3e8d0a9-b8fe-46c6-b1d5-99e033396013',
  'azureml.git.repository_uri': 'https://github.com/jtobelem-simplon/prepa-dp100.git',
  'mlflow.source.git.repoURL': 'https://github.com/jtobelem-simplon/prepa-dp100.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'ec3f4315d10480b95de0e7f372ca9be3530138bb',
  'mlflow.source.git.commit': 'ec3f4315d10480b95de0e7f372ca9be3530138bb',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'titanic-data.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_1a8c824b447c433cac6abf5cbe9e3489'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'Non

# Avec les datasets

In [22]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'titanic train dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/titanic/train.csv'], # Upload the titanic csv files in /data
                        target_path='titanic-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/train.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='titanic train dataset',
                                description='titanic training data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Uploading an estimated of 1 files
Uploading ./data/titanic/train.csv
Uploaded ./data/titanic/train.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered.


In [23]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'titanic test dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/titanic/test.csv'], # Upload the titanic csv files in /data
                        target_path='titanic-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/test.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='titanic test dataset',
                                description='titanic testing data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Uploading an estimated of 1 files
Uploading ./data/titanic/test.csv
Uploaded ./data/titanic/test.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered.


In [ ]:
titanic_ds = ws.datasets.get("titanic train dataset")
titanic_ds.to_pandas_dataframe().head()

In [25]:
# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



/home/lab/anaconda3/envs/azure/lib/python3.8/site-packages/azureml/dataprep/api/_dataframereader.py:234: UserWarning: Using alternate reader. Inconsistent or mixed schemas detected across partitions: partition had different number of columns. The first partition has 11 columns. Found partition has 12 columns.
First partition columns (ordered): ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
Found Partition has columns (ordered): ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Column12']
  warnings.warn('Using alternate reader. ' + reason)
/home/lab/anaconda3/envs/azure/lib/python3.8/site-packages/azureml/dataprep/api/_dataframereader.py:178: UserWarning: Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.12.0 --upgrade
  warnings.warn('Please install pyarrow>=0.1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Column12
0,892,3,"Kelly, Mr. James",male,34.5,0.0,0,330911,7.8292,None,Q,None
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1.0,0,363272,7.0000,None,S,None
2,894,2,"Myles, Mr. Thomas Francis",male,62,0.0,0,240276,9.6875,None,Q,None
3,895,3,"Wirz, Mr. Albert",male,27,0.0,0,315154,8.6625,None,S,None
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1.0,1,3101298,12.2875,None,S,None


# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)